In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from qgsw.utils.sorting import sort_files

from qgsw.run_summary import RunSummary
import torch
import matplotlib.pyplot as plt

ROOT = Path(os.path.abspath('')).parent

In [ ]:
plots = [
{
"field": "pv",
"layer": 0,
"input": "one_layer_baroclinic_30km",
"output": "1L_baroclinic_30km",
"steps": [33084, 62492, 91898],
"colorscale": "RdBu_r",
},
{
"field": "pv",
"layer": 0,
"input": "two_layers_baroclinic_30km",
"output": "2L_baroclinic_30km",
"steps": [33084, 62492, 91898],
"colorscale": "RdBu_r",
},
{
"field": "pv",
"layer": 1,
"input": "two_layers_baroclinic_30km",
"output": "2L_baroclinic_30km_bottom",
"steps": [33084, 62492, 91898],
"colorscale": "RdBu_r",
},
{
"field": "pv",
"layer": 0,
"input": "one_layer_baroclinic_100km",
"output": "1L_baroclinic_100km",
"steps": [33084, 62492, 91898],
"colorscale": "RdBu_r",
},
{
"field": "pv",
"layer": 0,
"input": "two_layers_baroclinic_100km",
"output": "2L_baroclinic_100km",
"steps": [33084, 62492, 91898],
"colorscale": "RdBu_r",
},
{
"field": "pv",
"layer": 1,
"input": "two_layers_baroclinic_100km",
"output": "2L_baroclinic_100km_bottom",
"steps": [33084, 62492, 91898],
"colorscale": "RdBu_r",
},
{
"field": "pv",
"layer": 0,
"input": "one_layer_barotropic_100km",
"output": "1L_barotropic_100km",
"steps": [33084, 62492, 91898],
"colorscale": "RdBu_r",
},
{
"field": "pv",
"layer": 0,
"input": "two_layers_barotropic_100km",
"output": "2L_barotropic_100km",
"steps": [33084, 62492, 91898],
"colorscale": "RdBu_r",},
{
"field": "pv",
"layer": 1,
"input": "two_layers_barotropic_100km",
"output": "2L_barotropic_100km_bottom",
"steps": [33084, 62492, 91898],
"colorscale": "RdBu_r",
}
]

In [ ]:
import os
from pathlib import Path

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from qgsw.run_summary import RunSummary
import toml

ROOT = Path(os.path.abspath('')).parent


plots_config = toml.load(ROOT.joinpath("config/save_plots.toml"))

for plot_config in plots:
    field = plot_config["field"]
    layer = plot_config["layer"]

    input_folder = ROOT.joinpath(f"output/g5k/{plot_config['input']}")
    output_folder = ROOT.joinpath(f"output/snapshots/{plot_config['output']}")
    if not output_folder.is_dir():
        output_folder.mkdir(parents=True)

    summary = RunSummary.from_file(input_folder.joinpath("_summary.toml"))
    config = summary.configuration
    x_min, x_max = config.space.box.x_min, config.space.box.x_max
    y_min, y_max = config.space.box.y_min, config.space.box.y_max
    
    offset = 24

    datas = []

    for i,step in enumerate(plot_config["steps"]):
        file = input_folder.joinpath(f"{config.model.prefix}{step}.npz")

        data = np.load(file)[field][0, layer, ...][offset:-offset,offset:-offset]

        datas.append(data)

    zmax = max(np.max(np.abs(data)) for data in datas)
    zmin = -zmax

    for i,data in enumerate(datas):
        colorbar = go.heatmap.ColorBar(
            exponentformat="e",
            showexponent="all",
            title={"text": "Potential Vorticity (s⁻¹)", "side": "right"},
            thickness=100
        )

        x = np.linspace(x_min / 1000, x_max / 1000, config.space.nx)[offset:-offset]
        y = np.linspace(y_min / 1000, y_max / 1000, config.space.ny)[offset:-offset]

        heatmap = go.Heatmap(
            z=data.T,
            x=x,
            y=y,
            colorscale=px.colors.diverging.RdBu_r,
            zmin=zmin,
            zmax=zmax,
            colorbar=colorbar,
            showscale=i == 2,
        )
        fig = go.Figure()
        fig.add_trace(heatmap)#, row=1, col=i+1)
        fig.add_shape(type="rect", xref=f"x", yref=f"y", x0=x[0], x1=x[-1], y0=y[0], y1=y[-1], line=dict(color="black", width=2))

        fig.update_layout(
            # template="simple_white",
            autosize=True,
            margin=dict(l=20, r=20, t=20, b=20),
            width=1000+ (400 * (i==2)) + 100 * (i==0),
            height=1000 ,
            font={"size": 60, "color":"black"},
            xaxis={"scaleanchor": "y", "constrain": "domain"},
            yaxis={"scaleanchor": "x", "constrain": "domain"},
        )
        fig.update_layout(
        )

        fig.update_xaxes(
            title={"text": "X (km)"},
            exponentformat="none",
            dtick = (x_max - x_min) / 1000 / 6 - ((x_max - x_min)/ 1000 / 6 )% 50,
            tick0 = 0,
        )

        fig.update_yaxes(
            title={"text": "Y (km)"},
            exponentformat="none",
            dtick = (y_max - y_min) / 1000 / 6 - ((y_max - y_min)/1000/6) % 50,
            tick0 = 0,
            ticksuffix = "  ",
            visible = i == 0,
        )
        # fig.show()
        # fig.write_image(output_folder.joinpath(f"snapshot_{plot_config['steps'][i]}.png"))

In [ ]:
plots_compare = [
{
"field": "pv",
"layer": 0,
"name": "1l vs 2l baroclinic",
"input": ["one_layer_baroclinic_30km", "two_layers_baroclinic_30km"],
"output": "compare_baroclinic_30km",
},
{
"field": "pv",
"layer": 0,
"name": "1l vs 2l barotropic",
"input": ["one_layer_barotropic_100km", "two_layers_barotropic_100km"],
"output": "compare_barotropic_100km",
},
{
"field": "pv",
"layer": 0,
"name": "1l vs 2l",
"input": ["one_layer_baroclinic_100km", "two_layers_baroclinic_100km"],
"output": "compare_baroclinic_100km",
},
{
"field": "pv",
"layer": 0,
"name": "SF0 vs 2l baroclinic",
"input": ["sf_baroclinic", "two_layers_baroclinic_30km"],
"output": "compare_baroclinic_sf",
},
{
"field": "pv",
"layer": 0,
"name": "PV0 vs 2l baroclinic",
"input": ["pv_baroclinic", "two_layers_baroclinic_30km"],
"output": "compare_baroclinic_pv",
},
{
"field": "pv",
"layer": 0,
"name": "SF1 vs 2l barotropic",
"input": ["sf_barotropic", "two_layers_barotropic_100km"],
"output": "compare_barotropic_sf",
},
{
"field": "pv",
"layer": 0,
"name": "PV1 vs 2l barotropic",
"input": ["pv_barotropic", "two_layers_barotropic_100km"],
"output": "compare_barotropic_pv",
},
{
"field": "pv",
"layer": 0,
"name": "SF0 vs 2l",
"input": ["sf_alpha_0", "two_layers_baroclinic_100km"],
"output": "compare_2l_sf",
},
{
"field": "pv",
"layer": 0,
"name": "SF0.1 vs 2l",
"input": ["sf_alpha_0_1", "two_layers_baroclinic_100km"],
"output": "compare_2l_sf_0_1",
},
{
"field": "pv",
"layer": 0,
"name": "SF0.2 vs 2l",
"input": ["sf_alpha_0_2", "two_layers_baroclinic_100km"],
"output": "compare_2l_sf_0_2",
},
{
"field": "pv",
"layer": 0,
"name": "SF0.3 vs 2l",
"input": ["sf_alpha_0_3", "two_layers_baroclinic_100km"],
"output": "compare_2l_sf_0_3",
},
{
"field": "pv",
"layer": 0,
"name": "SF0.5 vs 2l",
"input": ["sf_alpha_0_5", "two_layers_baroclinic_100km"],
"output": "compare_2l_sf_0_5",
},
{
"field": "pv",
"layer": 0,
"name": "SF0.15vs 2l",
"input": ["sf_alpha_0_15", "two_layers_baroclinic_100km"],
"output": "compare_2l_sf_0_15",
},
{
"field": "pv",
"layer": 0,
"name": "SF Changing 0.8 vs 2l",
"input": ["sf_alpha_changing", "two_layers_baroclinic_100km"],
"output": "compare_2l_sf_changing",
},
{
"field": "pv",
"layer": 0,
"name": "SF Changing 0.2 vs 2l",
"input": ["sf_alpha_0_2_changing", "two_layers_baroclinic_100km"],
"output": "compare_2l_sf_changing_0_2",
},
]

In [ ]:
def loss(x:np.ndarray, y:np.ndarray) -> float:
    return np.sqrt(np.mean(np.square(x - y))) / 9.375e-5

In [ ]:
import os
from pathlib import Path

import numpy as np
import plotly.graph_objects as go
from qgsw.run_summary import RunSummary
import toml
from qgsw.utils.sorting import sort_files

ROOT = Path(os.path.abspath('')).parent

plots_config = toml.load(ROOT.joinpath("config/save_plots.toml"))
fig = go.Figure()
for plot_config in plots_compare:
    field = plot_config["field"]
    layer = plot_config["layer"]

    input_folder1 = ROOT.joinpath(f"output/g5k/{plot_config['input'][0]}")
    input_folder2 = ROOT.joinpath(f"output/g5k/{plot_config['input'][1]}")
    output_folder = ROOT.joinpath(f"output/snapshots/{plot_config['output']}")

    if not output_folder.is_dir():
        output_folder.mkdir(parents=True)

    summary1 = RunSummary.from_file(input_folder1.joinpath("_summary.toml"))
    config1 = summary1.configuration
    summary2 = RunSummary.from_file(input_folder2.joinpath("_summary.toml"))
    config2 = summary2.configuration

    steps_1, files_1 = sort_files(list(input_folder1.glob(f"{config1.model.prefix}*.npz")),config1.model.prefix,".npz")
    steps_2, files_2 = sort_files(list(input_folder2.glob(f"{config2.model.prefix}*.npz")),config2.model.prefix,".npz")

    x_min, x_max = config1.space.box.x_min, config1.space.box.x_max
    y_min, y_max = config1.space.box.y_min, config1.space.box.y_max
    
    offset = 0

    losses = []
    times = []

    for i,file1 in enumerate(files_1):

        file2 = files_2[i]

        data1 = np.load(file1)[field][0, layer, ...]
        data2 = np.load(file2)[field][0, layer, ...]

        if offset != 0 :
            data1 = data1[offset:-offset,offset:-offset]
            data2 = data2[offset:-offset,offset:-offset]

        losses.append(loss(data1, data2))
        times.append(steps_1[i] * config1.simulation.dt / 3600 / 24)
    
    scatter = go.Scatter(
        x=times, 
        y=losses,
        name = plot_config["name"]
    )


    fig.add_trace(scatter)#, row=1, col=i+1)

fig.update_layout(
    # template="simple_white",
    autosize=True,
    width=1500,
    height=750 ,
    font={"size": 20, "color":"black"},
)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.update_xaxes(
    title={"text": "Time (day)"},
    exponentformat="e",
)

fig.update_yaxes(
    title={"text": "RMSE (normalized)"},
    exponentformat="none",
    ticksuffix = "  ",
)
# fig.show()

In [ ]:
import torch


baroclinic_30 = "two_layers_baroclinic_30km"
barotropic_100 = "two_layers_barotropic_100km"
baroclinic_100 = "two_layers_baroclinic_100km_long"

path = ROOT.joinpath(f"output/g5k/{baroclinic_30}")
run = RunSummary.from_file(path.joinpath("_summary.toml"))
steps, files_baroclinic_30 = sort_files(list(path.glob(f"{run.configuration.model.prefix}*.npz")),run.configuration.model.prefix,".npz")

path = ROOT.joinpath(f"output/g5k/{barotropic_100}")
run = RunSummary.from_file(path.joinpath("_summary.toml"))
steps, files_barotropic_100 = sort_files(list(path.glob(f"{run.configuration.model.prefix}*.npz")),run.configuration.model.prefix,".npz")

path = ROOT.joinpath(f"output/g5k/{baroclinic_100}")
run = RunSummary.from_file(path.joinpath("_summary.toml"))
steps, files_baroclinic_100 = sort_files(list(path.glob(f"{run.configuration.model.prefix}*.npz")),run.configuration.model.prefix,".npz")

all_files = [
    files_baroclinic_30,files_barotropic_100, files_baroclinic_100
]
names = ["baroclinic_30km", "barotropic_100km", "baroclinic_100km"]

H1,H2 = run.configuration.model.h
g1, g2 = run.configuration.model.g_prime

A = torch.tensor(
    [
        [1/H1/g1+1/H1/g2, -1/H1/g2],
        [-1/H2/g2, 1/H2/g2]
    ],
    dtype=torch.float64
)

P = torch.linalg.eig(A)[1].real

offset = 24



for j,files in enumerate(all_files):
    modes_1 = []
    modes_2 = []
    times=[]
    for i in range(0,len(steps),1):

        psi = torch.tensor(np.load(files[i])["p"], dtype=torch.float64)[...,offset:-offset,offset:-offset]
        modes = torch.einsum("lm,...mxy->...lxy", torch.inverse(P), psi)
        top = modes[0,0,...]
        bottom = modes[0,1,...]

        mode_1 = torch.mean(torch.abs(top))
        mode_2 = torch.mean(torch.abs(bottom))

        modes_1.append(mode_1 / (mode_1 + mode_2))
        modes_2.append(mode_2 / (mode_1 + mode_2))

        times.append(steps[i] * run.configuration.simulation.dt / 3600 / 24)

    fig = go.Figure()
    fig.update_layout(
        template="plotly_white",
        autosize=False,
        margin=dict(l=250 * (j==0) + 20, r=20, t=20, b=20),
        width=1000+ (525 * (j==2)) + 200 * (j==0),
        height=1000 ,
        font={"size": 60, "color":"black"},
        # legend=dict(orientation="h", x=0.1, y=0.9)
    )
    fig.update_yaxes(
        range=[-0.1,1.1],
        title = {"text": "Mode Intensity"},
        ticksuffix = "  ",
        tick0=0,
        dtick=0.25,
        visible = j == 0,
    )
    fig.update_xaxes(
        range=[times[0], times[-1]],
        title = {"text": "Time (day)"}
    )
    fig.add_shape(
        type="rect", xref=f"x", yref=f"y", x0=times[0], x1=times[-1], y0=-0.1, y1=1.1, line=dict(color="black", width=3)
    )
    fig.add_trace(
        go.Scatter(
            name = "Baroclinic Mode",
            x = times,
            y = modes_1,
            mode = "lines",
            line= dict(width=10),
            showlegend=(j==2)
        ),
    )
    fig.add_trace(
        go.Scatter(
            name="Barotropic Mode",
            x = times,
            y = modes_2,
            mode = "lines",
            line= dict(width=10),
            showlegend=(j==2)
        ),
    )
    # fig.show()
    # fig.write_image(f"../output/snapshots/modes/{names[j]}.png")

In [ ]:
folder = ROOT.joinpath("output/g5k/two_layers_baroclinic_30km")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))
model = run.configuration.model
steps, files_baroclinic = sort_files(folder.glob(f"{model.prefix}*.npz"), prefix=model.prefix, suffix=".npz")

folder = ROOT.joinpath("output/g5k/two_layers_barotropic_100km")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))
model = run.configuration.model
steps, files_barotropic = sort_files(folder.glob(f"{model.prefix}*.npz"), prefix=model.prefix, suffix=".npz")

folder = ROOT.joinpath("output/g5k/two_layers_baroclinic_100km_long")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))
model = run.configuration.model
steps, files_mix = sort_files(folder.glob(f"{model.prefix}*.npz"), prefix=model.prefix, suffix=".npz")

all_files = [files_baroclinic, files_barotropic, files_mix]

names = ["baroclinic", "barotropic", "mix"][-1:]

threshold = 20000


fig = go.Figure()
fig.update_yaxes(range=[-0.1,1.1])
for j,files in enumerate(all_files[-1:]):
    ratios = []
    times= []
    for i, file in enumerate(files):

        psi = torch.tensor(np.load(file)["p"], dtype=torch.float64) / 9.375e-5

        psi_top = psi[0,0,...]
        psi_bot = psi[0,1,...]

        above_thresh = torch.abs(psi_top) > threshold 

        ratios.append(torch.mean(psi_bot[above_thresh] / psi_top[above_thresh]))
        times.append(steps[i] * run.configuration.simulation.dt)

    
    fig.add_trace(go.Scatter(
        name=names[j],
        x=[time / 3600 / 24 for time in times],
        y=ratios,
        line=dict(width=3)
    ))
fig.show()

In [ ]:
np.load("../data/coefficients_0_2.npz")["alpha"]

In [ ]:
plt.plot(
    np.load("../data/coefficients_0_2_short.npz")["times"],
    np.load("../data/coefficients_0_2_short.npz")["alpha"]
)

In [ ]:
np.savez("../data/coefficients_0_2.npz", times = np.array(times), alpha=np.array([e.cpu().item() for e in ratios]))

In [ ]:
np.array(times)

In [ ]:
%matplotlib tk

import matplotlib.pyplot as plt

folder = ROOT.joinpath("output/g5k/two_layers_baroclinic_30km")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))
model = run.configuration.model
steps, files_baroclinic = sort_files(folder.glob(f"{model.prefix}*.npz"), prefix=model.prefix, suffix=".npz")

folder = ROOT.joinpath("output/g5k/two_layers_barotropic_100km")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))
model = run.configuration.model
steps, files_barotropic = sort_files(folder.glob(f"{model.prefix}*.npz"), prefix=model.prefix, suffix=".npz")

folder = ROOT.joinpath("output/g5k/two_layers_baroclinic_100km")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))
model = run.configuration.model
steps, files_mix = sort_files(folder.glob(f"{model.prefix}*.npz"), prefix=model.prefix, suffix=".npz")

all_files = [files_baroclinic, files_barotropic, files_mix]

names = ["baroclinic", "barotropic", "mix"]

threshold = 100

plt.close()
plt.ion()

for j,files in enumerate(all_files[-1:]):
    ratios = []
    times= []
    for i, file in enumerate(files):

        plt.cla()
        plt.title(f"{steps[i]} / {steps[-1]}")

        psi = torch.tensor(np.load(file)["p"], dtype=torch.float64) / 9.375e-5

        psi_top = psi[0,0,...]
        psi_bot = psi[0,1,...]

        above_thresh = torch.abs(psi_top) > threshold

        # ratios.append(torch.mean(psi_bot[above_thresh] / psi_top[above_thresh]))

        plt.hist(psi_bot[above_thresh] / psi_top[above_thresh],bins=300)
        plt.xlim((-3,3))

        # times.append(steps[i] * run.configuration.simulation.dt)

        plt.pause(0.1)
plt.ioff()